In [6]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import model_selection
import numpy as np
from sklearn.metrics import accuracy_score

Tras probar con varios modelos, he decido optar por un modelo de regresión logísitica y convertirlo a formato de multiclase. Por dos principales motivos, es el que me ha demostrado ser más consistente en sus resultados y no verse afectado por como se distribuyen los datos entre test y target, obteniendo valores muy similares en cada prueba. El segundo motivo y principal, es el que mayor promedio de acierto tiene. 

In [7]:
# Nos descargamos el Dataset con los datos de las temporadas 2000-01 hasta la temporada 2020-21

Dataset = pd.read_csv('Raw/DF_Completo.csv')

In [8]:
Dataset 

,Temporada,Fecha,Jornada,Equipo Local,Equipo Visitante,Resultado,Goles Local,Goles Visitante,Penaltis Local,Target,...,Pr G AF L,Pr G AF V,Pr G EC L,Pr G EC V,Pr TA L,Pr TA V,Pr TR L,Pr TR V,Pr TA ARB,Pr TR ARB
0,1,09/09/2000,1,Barcelona,Málaga,2-1,2,1,0,L,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1,1,09/09/2000,1,DeportivodeLaCoruña,AthleticClub,2-0,2,0,1,L,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2,1,09/09/2000,1,RealSociedad,RacingdeSantander,2-2,2,2,1,E,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
3,1,09/09/2000,1,RealMadrid,Valencia,2-1,2,1,0,L,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
4,1,09/09/2000,1,Zaragoza,Espanyol,1-2,1,2,0,V,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7975,21,22/05/2021,38,Osasuna,RealSociedad,0-1,0,1,0,V,...,1.167,1.333,1.222,0.944,2.135,2.216,0.054,0.027,4.000,0.118
7976,21,22/05/2021,38,RealMadrid,Villarreal,2-1,2,1,0,L,...,1.722,1.667,0.667,1.056,1.595,1.811,0.027,0.108,2.842,0.105
7977,21,22/05/2021,38,Valladolid,AtléticodeMadrid,1-2,1,2,0,V,...,1.000,1.333,1.556,0.722,2.432,2.541,0.108,0.000,5.706,0.059
7978,21,22/05/2021,38,Elche,AthleticClub,2-0,2,0,0,L,...,0.889,0.944,1.278,1.167,2.595,2.243,0.027,0.054,5.450,0.150


In [9]:
# Nos creamos la variables target donde obtendremos una columna con tres posibles resultados: L, E o V. 
# Según gana Local, empata el partido o gana el equipo visitante, respectivamente. 

def crear_target(df):
    Target = []
    for a in range(len(df)): 
        if df.loc[a,'Goles Local'] > df.loc[a,'Goles Visitante']:
            Target.append('L')
        elif df.loc[a,'Goles Local'] == df.loc[a,'Goles Visitante']:
            Target.append('E')
        else: 
            Target.append('V')
    df.insert(9,'Target',Target,True)
    return df

In [10]:
# Modificamos los nombres de las temporadas para tener un mayor entendimiento. 

ditc_temporada={1:'2000-01',
2:'2001-02',
3:'2002-03',
4:'2003-04',
5:'2004-05',
6:'2005-06',
7:'2006-07',
8:'2007-08',
9:'2008-09',
10:'2009-10',
11:'2010-11',
12:'2011-12',
13:'2012-13',
14:'2013-14',
15:'2014-15',
16:'2015-16',
17:'2016-17',
18:'2017-18',
19:'2018-19',
20:'2019-20',
21:'2020-21',
22:'2021-22'}

In [11]:
Dataset['Temporada'] =  Dataset['Temporada'].replace(ditc_temporada)

In [12]:
# Nos cargamos aquellas columnas que no vamos a emplear para predecir el resultado final. 

Dataset_practica = Dataset.drop(columns=['Fecha',
       'Resultado', 'Goles Local', 'Goles Visitante', 'Penaltis Local', 'Penaltis Visitante', 'Arbitro', 'Amarillas Local',
       'Amarillas Visitante', 'Rojas Local', 'Rojas Visitante',
       'Minutos Goles L', 'Goleadores L', 'Minutos Goles V', 'Goleadores V','Pr TA L', 'Pr TA V', 'Pr TR L', 'Pr TR V', 'Pr TA ARB',
       'Pr TR ARB'])

In [13]:
# La siguientes listas hacen referencia al promedio de puntos de cada equipo tanto como local como visitante. 
lista_pr_L = round(Dataset_practica['PTOS L']/(Dataset_practica['Jornada']-1),3)
lista_pr_V = round(Dataset_practica['PTOS V']/(Dataset_practica['Jornada']-1),3)

In [14]:
Dataset_practica

,Temporada,Jornada,Equipo Local,Equipo Visitante,Target,PTOS L,PTOS V,POSC L,POSC V,FORMA L,FORMA V,Pr PTOS L,Pr PTOS V,Pr G AF L,Pr G AF V,Pr G EC L,Pr G EC V
0,2000-01,1,Barcelona,Málaga,L,0,0,0,0,0.0,0.0,0.000,0.000,0.000,0.000,0.000,0.000
1,2000-01,1,DeportivodeLaCoruña,AthleticClub,L,0,0,0,0,0.0,0.0,0.000,0.000,0.000,0.000,0.000,0.000
2,2000-01,1,RealSociedad,RacingdeSantander,E,0,0,0,0,0.0,0.0,0.000,0.000,0.000,0.000,0.000,0.000
3,2000-01,1,RealMadrid,Valencia,L,0,0,0,0,0.0,0.0,0.000,0.000,0.000,0.000,0.000,0.000
4,2000-01,1,Zaragoza,Espanyol,V,0,0,0,0,0.0,0.0,0.000,0.000,0.000,0.000,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7975,2020-21,38,Osasuna,RealSociedad,V,44,59,11,5,7.0,6.0,0.481,0.481,1.167,1.333,1.222,0.944
7976,2020-21,38,RealMadrid,Villarreal,L,81,58,2,7,11.0,12.0,0.722,0.519,1.722,1.667,0.667,1.056
7977,2020-21,38,Valladolid,AtléticodeMadrid,V,31,83,19,1,2.0,13.0,0.296,0.648,1.000,1.333,1.556,0.722
7978,2020-21,38,Elche,AthleticClub,L,33,46,18,9,3.0,5.0,0.370,0.296,0.889,0.944,1.278,1.167


In [15]:
""" Extraemos del dataset las primeras diez jornadas porque consideramos que hasta entonces, los equipos no están consolidados en la tabla y 
 algunas variables dejan de tener sentido al principio de la liga. También eliminamos las últimas dos, dado que las motivaciones de los equipos
 pueden verse alteradas con respecto al tramo medio de la liga. Con esto me refiero a que algunas se juegan mucho y otros, nada."""

Dataset_practica = Dataset_practica[(Dataset_practica['Jornada']>10)&(Dataset_practica['Jornada']<37)]
Dataset_practica.drop(columns=['PTOS L', 'PTOS V'], inplace=True)

""" Debido a que el 2019 fue un año atípico por el parón liguero por el covid, lo extraemos del dataset."""

Dataset_practica[Dataset_practica['Temporada']!='2019-20']


/var/folders/xg/mm8z7nf57pbbnsfb9kynnv4m0000gn/T/ipykernel_6669/134368943.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Dataset_practica.drop(columns=['PTOS L', 'PTOS V'], inplace=True)


,Temporada,Jornada,Equipo Local,Equipo Visitante,Target,POSC L,POSC V,FORMA L,FORMA V,Pr PTOS L,Pr PTOS V,Pr G AF L,Pr G AF V,Pr G EC L,Pr G EC V
100,2000-01,11,Zaragoza,Barcelona,L,17,7,7.0,7.0,0.400,0.400,1.200,1.600,1.000,1.400
101,2000-01,11,Oviedo,Valladolid,L,10,11,9.0,5.0,0.867,0.200,2.400,0.800,1.000,1.200
102,2000-01,11,RayoVallecano,Málaga,L,8,14,7.0,8.0,0.667,0.267,1.800,0.800,0.600,1.200
103,2000-01,11,RealSociedad,Numancia,L,19,15,3.0,4.0,0.133,0.267,0.800,1.400,2.800,2.000
104,2000-01,11,Espanyol,AthleticClub,L,12,13,8.0,5.0,0.467,0.200,1.000,0.400,0.800,1.400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7955,2020-21,36,CeltadeVigo,Getafe,L,8,15,10.0,4.0,0.529,0.255,1.588,0.765,1.588,1.706
7956,2020-21,36,AtléticodeMadrid,RealSociedad,L,1,5,10.0,9.0,0.824,0.510,2.176,1.353,0.529,0.882
7957,2020-21,36,Elche,Alavés,V,18,16,4.0,8.0,0.392,0.216,0.941,0.765,1.235,1.824
7958,2020-21,36,Osasuna,Cádiz,L,12,11,7.0,8.0,0.451,0.451,1.059,0.882,1.176,1.353


In [16]:
Dataset_practica['Pr liga L'] = lista_pr_L
Dataset_practica['Pr liga V'] = lista_pr_V

/var/folders/xg/mm8z7nf57pbbnsfb9kynnv4m0000gn/T/ipykernel_6669/598008605.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Dataset_practica['Pr liga L'] = lista_pr_L
/var/folders/xg/mm8z7nf57pbbnsfb9kynnv4m0000gn/T/ipykernel_6669/598008605.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Dataset_practica['Pr liga V'] = lista_pr_V


In [17]:
Dataset_practica2 = Dataset_practica.copy()

In [18]:
Dataset_practica2

,Temporada,Jornada,Equipo Local,Equipo Visitante,Target,POSC L,POSC V,FORMA L,FORMA V,Pr PTOS L,Pr PTOS V,Pr G AF L,Pr G AF V,Pr G EC L,Pr G EC V,Pr liga L,Pr liga V
100,2000-01,11,Zaragoza,Barcelona,L,17,7,7.0,7.0,0.400,0.400,1.200,1.600,1.000,1.400,1.000,1.600
101,2000-01,11,Oviedo,Valladolid,L,10,11,9.0,5.0,0.867,0.200,2.400,0.800,1.000,1.200,1.400,1.200
102,2000-01,11,RayoVallecano,Málaga,L,8,14,7.0,8.0,0.667,0.267,1.800,0.800,0.600,1.200,1.600,1.200
103,2000-01,11,RealSociedad,Numancia,L,19,15,3.0,4.0,0.133,0.267,0.800,1.400,2.800,2.000,0.800,1.100
104,2000-01,11,Espanyol,AthleticClub,L,12,13,8.0,5.0,0.467,0.200,1.000,0.400,0.800,1.400,1.200,1.200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7955,2020-21,36,CeltadeVigo,Getafe,L,8,15,10.0,4.0,0.529,0.255,1.588,0.765,1.588,1.706,1.343,0.971
7956,2020-21,36,AtléticodeMadrid,RealSociedad,L,1,5,10.0,9.0,0.824,0.510,2.176,1.353,0.529,0.882,2.200,1.600
7957,2020-21,36,Elche,Alavés,V,18,16,4.0,8.0,0.392,0.216,0.941,0.765,1.235,1.824,0.857,0.914
7958,2020-21,36,Osasuna,Cádiz,L,12,11,7.0,8.0,0.451,0.451,1.059,0.882,1.176,1.353,1.171,1.229


In [19]:
# Restamos aquellas entre Local y Visitante, para obtener una única columnas por parámetro que nos resuma la relación entre Local y Visitante. 
# Simplificamos el modelo. 

Jornada_list = Dataset_practica2['Jornada']
Data_pr_liga = Dataset_practica2['Pr liga L']-Dataset_practica2['Pr liga V']
Data_Local_list =Dataset_practica2['Equipo Local']
Data_Visit_list =Dataset_practica2['Equipo Visitante']
Difrencia_pos_lista = Dataset_practica2['POSC L'] - Dataset_practica2['POSC V']
Diferencia_forma = Dataset_practica2['FORMA L'] - Dataset_practica2['FORMA V']
Diferencia_pr_ = Dataset_practica2['Pr PTOS L'] - Dataset_practica2['Pr PTOS V']
Diferencia_pr_goles_af = Dataset_practica2['Pr G AF L'] - Dataset_practica2['Pr G AF V']
Diferencia_pr_goles_ec = Dataset_practica2['Pr G EC L'] - Dataset_practica2['Pr G EC  V']
Target_list = Dataset_practica2['Target']
Temporada_list = Dataset_practica2['Temporada']

In [20]:
# Obtenemos el DataFrame final 
Dataset_practica = pd.DataFrame(list(zip(Temporada_list,Jornada_list,Data_Local_list,Data_Visit_list,Difrencia_pos_lista,Diferencia_forma,Diferencia_pr_,Diferencia_pr_goles_af,Diferencia_pr_goles_ec,Target_list)), columns = ['Temporada','Jornada','Equipo Local','Equipo Visitante', 'Dif POS','Dif FORMA','Dif PTS L/V', 'Dif G AF', 'Dif G EC','Target'])
Dataset_practica

,Temporada,Jornada,Equipo Local,Equipo Visitante,Dif POS,Dif FORMA,Dif PTS L/V,Dif G AF,Dif G EC,Target
0,2000-01,11,Zaragoza,Barcelona,10,0.0,0.000,-0.400,-0.400,L
1,2000-01,11,Oviedo,Valladolid,-1,4.0,0.667,1.600,-0.200,L
2,2000-01,11,RayoVallecano,Málaga,-6,-1.0,0.400,1.000,-0.600,L
3,2000-01,11,RealSociedad,Numancia,4,-1.0,-0.134,-0.600,0.800,L
4,2000-01,11,Espanyol,AthleticClub,-1,3.0,0.267,0.600,-0.600,L
...,...,...,...,...,...,...,...,...,...,...
5455,2020-21,36,CeltadeVigo,Getafe,-7,6.0,0.274,0.823,-0.118,L
5456,2020-21,36,AtléticodeMadrid,RealSociedad,-4,1.0,0.314,0.823,-0.353,L
5457,2020-21,36,Elche,Alavés,2,-4.0,0.176,0.176,-0.589,V
5458,2020-21,36,Osasuna,Cádiz,1,-1.0,0.000,0.177,-0.177,L


In [21]:
# Este va a ser nuestro Test Real con el cual contrastar. 

Test_real = list(Dataset_practica.iloc[-1092:,9])

Como hemos comentado al principio, vamos a emplear una regresión logísitca donde crearemos un modelo para cada posible resultado y así obtener mediante la funcion predict_proba, la probabilidad del suceso para cada evento. Al final, compararemos las probabilidad y nos quedaremos con aquella que mayor probabilidad tenga. 

# Pr(Gana Local)

In [22]:
# Obtenemos X, eliminando aquellas columnas innecesarias y 
X = Dataset_practica.drop(columns=['Temporada', 'Jornada', 'Equipo Local', 'Equipo Visitante','Target'])


In [23]:
# Escalamos nuestros valores de X. 

from sklearn import preprocessing

std_scale = preprocessing.StandardScaler().fit(X)
X_train_scal = std_scale.transform(X)

In [24]:
dataframe_x_scale = pd.DataFrame(X_train_scal, columns=X.columns)
dataframe_x_scale

,Dif POS,Dif FORMA,Dif PTS L/V,Dif G AF,Dif G EC
0,1.188358,0.089180,-0.752280,-1.040888,0.033801
1,-0.146461,0.965730,1.862708,1.487824,0.351449
2,-0.753197,-0.129957,0.815929,0.729211,-0.283847
3,0.460275,-0.129957,-1.277630,-1.293759,1.939690
4,-0.146461,0.746592,0.294499,0.223468,-0.283847
...,...,...,...,...,...
5455,-0.874545,1.404005,0.321943,0.505420,0.481685
5456,-0.510503,0.308318,0.478764,0.505420,0.108448
5457,0.217580,-0.787370,-0.062268,-0.312619,-0.266377
5458,0.096233,-0.129957,-0.752280,-0.311354,0.387978


In [25]:
dataframe_x_scale.to_csv('processed/x_scaled.csv', index=False)

In [26]:
# Para Y, modificamos el resultado 'L' por 1 y el resto 0. 
y_df = pd.DataFrame(Dataset_practica,columns=['Target'])
y_df["Target"] = np.where(y_df["Target"] == 'L', 1, 0)
y_df.reset_index(drop=True, inplace=True)
y_l = y_df["Target"]


In [27]:
validation_size = 0.20
seed = 16
X_train_l, X_test_l, Y_train_l, Y_test_l = model_selection.train_test_split(dataframe_x_scale,
                                                                    y_l,
                                                                    test_size=validation_size,
                                                                    random_state=seed,
                                                                    shuffle=False)

In [28]:
from sklearn import linear_model


In [29]:
model = linear_model.LogisticRegression(max_iter = 1000)

In [30]:
name='Logistic Regression'
kfold = model_selection.KFold(n_splits=10, random_state=17, shuffle =True) #Parte los datos en 10 trozos para usar validación cruzada / cross validation
cv_results = model_selection.cross_val_score(model, X_train_l, Y_train_l, cv=kfold, scoring='accuracy')

msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
print(cv_results)
print(msg)

[0.63844394 0.61327231 0.57894737 0.54004577 0.60183066 0.63157895
 0.60183066 0.61784897 0.58715596 0.62385321]
Logistic Regression: 0.603481 (0.027617)


In [31]:
model.fit(X_train_l, Y_train_l)

LogisticRegression(max_iter=1000)

In [32]:
model.score(X_test_l, Y_test_l)

0.6510989010989011

In [84]:
import pickle

with open('model/my_model_l', 'wb') as archivo_salida:
    pickle.dump(model, archivo_salida)

In [33]:
prediction = model.predict(X_test_l)
prediction

array([1, 0, 0, ..., 0, 0, 0])

In [34]:
DataFrame_Prediction = pd.DataFrame(prediction, columns=['Predicciones'])
DataFrame_Prediction.Predicciones.value_counts()

0    645
1    447
Name: Predicciones, dtype: int64

In [35]:
# En este caso, la segunda fila sería la probabilidad de que gane el equipo Local. 
prediction_proba = model.predict_proba(X_test_l)
prediction_proba

array([[0.48979737, 0.51020263],
       [0.83318637, 0.16681363],
       [0.65266181, 0.34733819],
       ...,
       [0.55654647, 0.44345353],
       [0.56179545, 0.43820455],
       [0.79399377, 0.20600623]])

# Pr(Gana Visitante)

In [36]:
# Obtenemos X, eliminando aquellas columnas innecesarias y para Y, modificamos el resultado 'V' por 1 y el resto 0. 

X = Dataset_practica.drop(columns=['Temporada', 'Jornada', 'Equipo Local', 'Equipo Visitante','Target'])
y_df = pd.DataFrame(Dataset_practica,columns=['Target'])
y_df["Target"] = np.where(y_df["Target"] == 'V', 1, 0)
y_df.reset_index(drop=True, inplace=True)
y_v = y_df["Target"]
y_v

0       0
1       0
2       0
3       0
4       0
       ..
5455    0
5456    0
5457    1
5458    0
5459    0
Name: Target, Length: 5460, dtype: int64

In [37]:
validation_size = 0.20
seed = 0
X_train_v, X_test_v, Y_train_v, Y_test_v = model_selection.train_test_split(dataframe_x_scale,
                                                                    y_v,
                                                                    test_size=validation_size,
                                                                    random_state=seed,
                                                                    shuffle=False)

In [38]:
model_v = linear_model.LogisticRegression(max_iter = 1000)

In [39]:
name='Logistic Regression'
kfold = model_selection.KFold(n_splits=10, random_state=17, shuffle =True) #Parte los datos en 10 trozos para usar validación cruzada / cross validation
cv_results = model_selection.cross_val_score(model_v, X_train_v, Y_train_v, cv=kfold, scoring='accuracy')

msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
print(cv_results)
print(msg)

[0.73913043 0.73684211 0.71167048 0.75057208 0.76659039 0.72997712
 0.78718535 0.74828375 0.74770642 0.73853211]
Logistic Regression: 0.745649 (0.019417)


In [40]:
model_v.fit(X_train_v, Y_train_v)

LogisticRegression(max_iter=1000)

In [83]:
import pickle

with open('model/my_model_v', 'wb') as archivo_salida:
    pickle.dump(model_v, archivo_salida)

In [41]:
model_v.score(X_test_v, Y_test_v)

0.7243589743589743

In [42]:
# En este caso, la segunda fila sería la probabilidad de que gane el equipo visitante. 
prediction_proba = model_v.predict_proba(X_test_l)
prediction_proba

array([[0.74313247, 0.25686753],
       [0.39133135, 0.60866865],
       [0.61146684, 0.38853316],
       ...,
       [0.72377677, 0.27622323],
       [0.68836725, 0.31163275],
       [0.42507409, 0.57492591]])

# Pr(Empatan)

In [43]:
# Obtenemos X, eliminando aquellas columnas innecesarias y para Y, modificamos el resultado 'E' por 1 y el resto 0. 

X = Dataset_practica.drop(columns=['Temporada', 'Jornada', 'Equipo Local', 'Equipo Visitante','Target'])
y_df = pd.DataFrame(Dataset_practica,columns=['Target'])
y_df["Target"] = np.where(y_df["Target"] == 'E', 1, 0)
y_df.reset_index(drop=True, inplace=True)
y_e = y_df["Target"]
y_e

0       0
1       0
2       0
3       0
4       0
       ..
5455    0
5456    0
5457    0
5458    0
5459    1
Name: Target, Length: 5460, dtype: int64

In [44]:
validation_size = 0.20
seed = 16
X_train_E, X_test_E, Y_train_E, Y_test_E = model_selection.train_test_split(dataframe_x_scale,
                                                                    y_e,
                                                                    test_size=validation_size,
                                                                    random_state=seed,
                                                                    shuffle=False)

In [79]:
Y_train_E.to_csv('processed/Y_train_E.csv', index=False)

In [45]:
model_e = linear_model.LogisticRegression(max_iter = 1000)

In [46]:
name='Logistic Regression'
kfold = model_selection.KFold(n_splits=10, random_state=0, shuffle =True) #Parte los datos en 10 trozos para usar validación cruzada / cross validation
cv_results = model_selection.cross_val_score(model_e, X_train_E, Y_train_E, cv=kfold, scoring='accuracy')

msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
print(cv_results)
print(msg)

[0.74141876 0.75057208 0.75057208 0.77574371 0.72997712 0.78718535
 0.73684211 0.77116705 0.75688073 0.77293578]
Logistic Regression: 0.757329 (0.017827)


In [47]:
model_e.fit(X_train_E, Y_train_E)

LogisticRegression(max_iter=1000)

In [82]:
import pickle

with open('model/my_model_e', 'wb') as archivo_salida:
    pickle.dump(model_e, archivo_salida)

In [48]:
model_e.score(X_test_E, Y_test_E)

0.7335164835164835

In [49]:
# En este caso, la segunda fila sería la probabilidad de que el partido termine en empate. 
prediction_proba = model_e.predict_proba(X_test_E)
prediction_proba

array([[0.79079917, 0.20920083],
       [0.66960287, 0.33039713],
       [0.73942342, 0.26057658],
       ...,
       [0.7414366 , 0.2585634 ],
       [0.77375806, 0.22624194],
       [0.71800835, 0.28199165]])

# Unificamos probabilidades en una misma tabla

In [50]:
Dataframe_proba = pd.DataFrame(list(zip(model.predict_proba(X_test_E)[:,1],model_e.predict_proba(X_test_E)[:,1],model_v.predict_proba(X_test_E)[:,1])),columns=['Local','Empate','Visitante'])
Dataframe_proba

,Local,Empate,Visitante
0,0.510203,0.209201,0.256868
1,0.166814,0.330397,0.608669
2,0.347338,0.260577,0.388533
3,0.298855,0.254620,0.455767
4,0.371505,0.278527,0.336702
...,...,...,...
1087,0.559801,0.212818,0.212267
1088,0.551438,0.227933,0.206001
1089,0.443454,0.258563,0.276223
1090,0.438205,0.226242,0.311633


In [51]:
# Creamos una columna con aquel suceso que mayor probabilidad. 

pronóstico_list = []
for i in range(len(Dataframe_proba)):
    L = Dataframe_proba.iloc[i,0]
    E = Dataframe_proba.iloc[i,1]
    V = Dataframe_proba.iloc[i,2]
    lista = [L,E,V]
    maximo = (max(lista))
    if lista.index(maximo) ==0: 
        pronóstico = 'L'
        pronóstico_list.append(pronóstico)

    elif lista.index(maximo) ==2: 
        pronóstico = 'V'
        pronóstico_list.append(pronóstico)

    elif lista.index(maximo) ==1: 
        pronóstico = 'E'
        pronóstico_list.append(pronóstico)

In [52]:
Dataframe_proba['Pronóstico'] = pronóstico_list
Dataframe_proba ['Real'] = Test_real
Dataframe_proba

,Local,Empate,Visitante,Pronóstico,Real
0,0.510203,0.209201,0.256868,L,L
1,0.166814,0.330397,0.608669,V,L
2,0.347338,0.260577,0.388533,V,E
3,0.298855,0.254620,0.455767,V,L
4,0.371505,0.278527,0.336702,L,V
...,...,...,...,...,...
1087,0.559801,0.212818,0.212267,L,L
1088,0.551438,0.227933,0.206001,L,L
1089,0.443454,0.258563,0.276223,L,V
1090,0.438205,0.226242,0.311633,L,L


In [53]:
Dataframe_proba['Pronóstico'].value_counts()

L    813
V    279
Name: Pronóstico, dtype: int64

Como vemos, a pesar de haber 3 opciones de resultado (Gana L,Gana V o Empatan), el modelo no ha indicado ningún encuentro como posible empate y en el 75% de los casos ha indicado que gana el equipo Local, algo que se podría considerar que no tiene mucha lógica. Esto tiene sentido y es algo que argumentaré en la presentación dado que está históricamente aproximadamente el 47% de los partidos gana el equipo Local, el 28% el visitante, y el 25% el empate. 

Además de este hecho, el empate no tiene un perfil de partido predefinido, a diferencia de como ocurre con 'Gana L' o 'Gana V', donde aquel equipo con mejor rendimiento suele ganar el partido. 

In [54]:
Dataset['Target'].value_counts()

L    3786
V    2201
E    1993
Name: Target, dtype: int64

In [55]:
print('% Victorias Local:', round(Dataset['Target'].value_counts()[0]/Dataset['Target'].value_counts().sum(),2))
print('% Victorias Visitante:', round(Dataset['Target'].value_counts()[1]/Dataset['Target'].value_counts().sum(),2))
print('% Victorias Empate:', round(Dataset['Target'].value_counts()[2]/Dataset['Target'].value_counts().sum(),2))

% Victorias Local: 0.47
% Victorias Visitante: 0.28
% Victorias Empate: 0.25


## ACCURACY - RESULTADO

In [56]:
accuracy_score(Test_real, pronóstico_list)

0.5018315018315018

Como vemos el nivel de acierto del modelo es muy bajo, y es algo que desde el principio me ha preocupado. Es por ello, que para tener 
una idea de lo razonable de este he querido contrastarlo con la probabilidad de acierto de una de las casas de apuestas más populares de nuestro país, el BET365. Véase a continuación: 

In [57]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context 

In [58]:
laligadf2021 = pd.read_csv('https://www.football-data.co.uk/mmz4281/2021/SP1.csv')
laligadf2021['Temporada']='2020-21'
laligadf1819 = pd.read_csv('https://www.football-data.co.uk/mmz4281/1819/SP1.csv')
laligadf1819['Temporada']='2018-19'
laligadf1718 = pd.read_csv('https://www.football-data.co.uk/mmz4281/1718/SP1.csv')
laligadf1718['Temporada']='2017-18'
laligadf1617 = pd.read_csv('https://www.football-data.co.uk/mmz4281/1617/SP1.csv')
laligadf1617['Temporada']='2016-17'
laligadf1516 = pd.read_csv('https://www.football-data.co.uk/mmz4281/1516/SP1.csv')
laligadf1516['Temporada']='2015-16'
laligadf1415 = pd.read_csv('https://www.football-data.co.uk/mmz4281/1415/SP1.csv')
laligadf1415['Temporada']='2014-15'
laligadf1314 = pd.read_csv('https://www.football-data.co.uk/mmz4281/1314/SP1.csv')
laligadf1314['Temporada']='2013-14'
laligadf1213 = pd.read_csv('https://www.football-data.co.uk/mmz4281/1213/SP1.csv')
laligadf1213['Temporada']='2012-13'
laligadf1112 = pd.read_csv('https://www.football-data.co.uk/mmz4281/1112/SP1.csv')
laligadf1112['Temporada']='2011-12'
laligadf1011 = pd.read_csv('https://www.football-data.co.uk/mmz4281/1011/SP1.csv')
laligadf1011['Temporada']='2010-11'
laligadf0910 = pd.read_csv('https://www.football-data.co.uk/mmz4281/0910/SP1.csv')
laligadf0910['Temporada']='2009-10'
laligadf0809= pd.read_csv('https://www.football-data.co.uk/mmz4281/0809/SP1.csv')
laligadf0809['Temporada']='2008-09'
laligadf0708 = pd.read_csv('https://www.football-data.co.uk/mmz4281/0708/SP1.csv')
laligadf0708['Temporada']='2007-08'
laligadf0607 = pd.read_csv('https://www.football-data.co.uk/mmz4281/0607/SP1.csv')
laligadf0607['Temporada']='2006-07'
laligadf0506 = pd.read_csv('https://www.football-data.co.uk/mmz4281/0506/SP1.csv')
laligadf0506['Temporada']='2005-06'

In [59]:
Databet = pd.concat([laligadf0506,laligadf0607,laligadf0708,laligadf0809,laligadf0910,laligadf1011,laligadf1112,laligadf1213,laligadf1314,laligadf1415,laligadf1516,laligadf1617,laligadf1718,laligadf1819,laligadf2021])
Databet.reset_index(drop=True, inplace=True)
Databet = Databet[['Temporada','HomeTeam','AwayTeam','FTR','B365H','B365D','B365A']]
Databet = Databet.rename(columns={"FTR": "Resultado"})
Databet

,Temporada,HomeTeam,AwayTeam,Resultado,B365H,B365D,B365A
0,2005-06,Alaves,Barcelona,D,7.00,3.75,1.50
1,2005-06,Ath Bilbao,Sociedad,H,2.00,3.25,3.25
2,2005-06,Valencia,Betis,H,2.00,3.25,3.25
3,2005-06,Ath Madrid,Zaragoza,D,1.72,3.40,4.00
4,2005-06,Cadiz,Real Madrid,A,7.50,4.00,1.44
...,...,...,...,...,...,...,...
5695,2020-21,Osasuna,Sociedad,A,5.00,4.33,1.61
5696,2020-21,Real Madrid,Villarreal,H,1.40,5.00,6.50
5697,2020-21,Valladolid,Ath Madrid,A,9.00,5.25,1.33
5698,2020-21,Granada,Getafe,D,2.62,3.40,2.60


Es necesario saber, por conocimiento del sector que aquellas cuota que se paga más bajo es el resultado que la casa da apuestas predice como mayor probabilidad de que ocurra. 

In [60]:
PREDICCION_BET = []
for a in range(len(Databet)): 
    if float(Databet.loc[a,'B365H']) < float(Databet.loc[a,'B365A']) and float(Databet.loc[a,'B365H']) < float(Databet.loc[a,'B365D']):
        PREDICCION_BET.append('H')
    elif float(Databet.loc[a,'B365H']) > float(Databet.loc[a,'B365A']) and float(Databet.loc[a,'B365A']) < float(Databet.loc[a,'B365D']):
        PREDICCION_BET.append('A')
    elif float(Databet.loc[a,'B365D']) < float(Databet.loc[a,'B365A']) and float(Databet.loc[a,'B365D']) < float(Databet.loc[a,'B365H']): 
        PREDICCION_BET.append('D')
    else:
        PREDICCION_BET.append('N/A')

In [61]:
Databet.insert(4, "PREDICCION BET", PREDICCION_BET, allow_duplicates=False)

In [62]:
Databet

,Temporada,HomeTeam,AwayTeam,Resultado,PREDICCION BET,B365H,B365D,B365A
0,2005-06,Alaves,Barcelona,D,A,7.00,3.75,1.50
1,2005-06,Ath Bilbao,Sociedad,H,H,2.00,3.25,3.25
2,2005-06,Valencia,Betis,H,H,2.00,3.25,3.25
3,2005-06,Ath Madrid,Zaragoza,D,H,1.72,3.40,4.00
4,2005-06,Cadiz,Real Madrid,A,A,7.50,4.00,1.44
...,...,...,...,...,...,...,...,...
5695,2020-21,Osasuna,Sociedad,A,A,5.00,4.33,1.61
5696,2020-21,Real Madrid,Villarreal,H,H,1.40,5.00,6.50
5697,2020-21,Valladolid,Ath Madrid,A,A,9.00,5.25,1.33
5698,2020-21,Granada,Getafe,D,A,2.62,3.40,2.60


In [63]:
ACIERTOS = []
for a in range(len(Databet)): 
    if Databet.loc[a,'Resultado'] == Databet.loc[a,'PREDICCION BET']:
        ACIERTOS.append(1)
    else:  
        ACIERTOS.append(0)

In [64]:
Databet.insert(5, "ACIERTOS", ACIERTOS, allow_duplicates=False)

In [65]:
Databet

,Temporada,HomeTeam,AwayTeam,Resultado,PREDICCION BET,ACIERTOS,B365H,B365D,B365A
0,2005-06,Alaves,Barcelona,D,A,0,7.00,3.75,1.50
1,2005-06,Ath Bilbao,Sociedad,H,H,1,2.00,3.25,3.25
2,2005-06,Valencia,Betis,H,H,1,2.00,3.25,3.25
3,2005-06,Ath Madrid,Zaragoza,D,H,0,1.72,3.40,4.00
4,2005-06,Cadiz,Real Madrid,A,A,1,7.50,4.00,1.44
...,...,...,...,...,...,...,...,...,...
5695,2020-21,Osasuna,Sociedad,A,A,1,5.00,4.33,1.61
5696,2020-21,Real Madrid,Villarreal,H,H,1,1.40,5.00,6.50
5697,2020-21,Valladolid,Ath Madrid,A,A,1,9.00,5.25,1.33
5698,2020-21,Granada,Getafe,D,A,0,2.62,3.40,2.60


In [66]:
print('% de acierto BET365:', Databet['ACIERTOS'].value_counts()[1]/Databet['ACIERTOS'].value_counts().sum())

% de acierto BET365: 0.539298245614035


Por si hubiera alguna duda, quisiera calcular el % de acierto para los últimos años (temporadas 2020-21 y 2018-19) dado que para los primeros años se entenderían unas tasas más bajas de aciertos dado que acaban de entrar en el mercado y sus algoritmos podría afectar esto a los pronósticos de sus modelos.

In [67]:
data_actual = Databet[(Databet['Temporada'] =='2020-21')|(Databet['Temporada'] =='2018-19')]
data_actual['ACIERTOS'].value_counts()

1    385
0    375
Name: ACIERTOS, dtype: int64

In [68]:
print('% de acierto BET365:', data_actual['ACIERTOS'].value_counts()[1]/data_actual['ACIERTOS'].value_counts().sum())

% de acierto BET365: 0.506578947368421


No obstante, vemos que los pronósticos siguen siendo aún más bajos y que la gran incertidumbre que hay en estos mercados lo hacen casi 'impredecible', aún para la casas de apuestas quienes llevan muchos años entrenando a sus modelos y tienen un mayor nivel de herramientas y datos para hacer sus modelos. Es por ello, que el 0,50 de accuraccy de mi modelo no queda tan atrás. 

Adicionalmente, vemos también como las se distribuye el pronóstico de BET365 entre Gana Local, Gana Visitante y Empate, para meditar el problema de estructuración de mis pronósticos. 

In [69]:
Databet['PREDICCION BET'].value_counts()

H      4211
A      1439
N/A      50
Name: PREDICCION BET, dtype: int64

N/A es el resultado cuando BET considera que dos posibles hecho tienen la misma probabilidad de suceso. En ese caso, no hemos estipulado que BET indica un pronóstico. Para el resto, este ha sido su distribucion: 


In [70]:
print('% de pronósticos Gana L', round(Databet['PREDICCION BET'].value_counts()[0]/Databet['PREDICCION BET'].value_counts().sum(),2))
print('% de pronósticos Gana V', round(Databet['PREDICCION BET'].value_counts()[1]/Databet['PREDICCION BET'].value_counts().sum(),2))

% de pronósticos Gana L 0.74
% de pronósticos Gana V 0.25


'N/A' apenas supone un 0.9 % de la muestra por lo que no lo considera significante para los cálculos anteriores. 

In [71]:
print('% N/A', round(Databet['PREDICCION BET'].value_counts()[2]/Databet['PREDICCION BET'].value_counts().sum(),3))

% N/A 0.009


Finalmente, como vemos BET365 tiene una distrubución de pronósticos y % de acierto muy similar al modelo simulado, aún no tomando los mismos datos, dado que los desconozco y los tomados por mi modelo han sido obtenido mediante Webscrapping y creando las columnas que debido a mi afición por este deporte he considerado que podrían ser significantes. La obtención del Dataset Completo viene recogido en la carpeta 'UTILS - DATASET WEBSCRAPPING'.   

En la presentación final, indagaré más sobre mi % de acierto y en técnicas para intentar elevar este % de acierto y sacar cierto beneficio a las casas de apuestas.